# Modelo Gradient Boosting Mejorado

In [ ]:
# Modelo Gradient Boosting optimizado para máximo rendimiento
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
# ------------------------  1  ---------------------------------------


Leer archivos de manera local

In [ ]:
# Cargar datos
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
# ------------------------  2  ---------------------------------------

Leer archivos desde Kaggle

In [ ]:
# Cargar datos
from google.colab import drive
drive.mount('/content/drive')
url = '1mMaUvQuLaEOcwekPww2RGsfZjxCaqT_q'

gdown.download(f"https://drive.google.com/uc?export=download&id={url}", 'kaggle.json', quiet=False)

os.makedirs("/root/.kaggle", exist_ok=True)

shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")

!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

zip_file_path = '/content/udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip'

if os.path.exists(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall()
# ------------------------  2  ---------------------------------------

# Proceso

In [ ]:
# Usar el 100% de los datos
train.drop(columns=["PERIODO"], inplace=True)
test.drop(columns=["PERIODO"], inplace=True)
# ------------------------  3  ---------------------------------------

train.fillna("Desconocido", inplace=True)
test.fillna("Desconocido", inplace=True)
# ------------------------  4  ---------------------------------------

# Codificación ordinal optimizada
cat_cols = train.select_dtypes(include='object').columns.drop("RENDIMIENTO_GLOBAL")
for col in cat_cols:
    orden = train[col].astype("category").cat.categories
    train[col] = train[col].astype("category").cat.codes
    test[col] = pd.Categorical(test[col], categories=orden).codes
# ------------------------  5  ---------------------------------------

# Variable objetivo
X = train.drop(columns=["RENDIMIENTO_GLOBAL"])
y = train["RENDIMIENTO_GLOBAL"]
le = LabelEncoder()
y_encoded = le.fit_transform(y)
# ------------------------  6  ---------------------------------------

# Entrenamiento sin validación para máximo ajuste
model = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    min_samples_split=4,
    min_samples_leaf=3,
    subsample=0.9,
    max_features='sqrt',
    random_state=42
)
model.fit(X, y_encoded)
# ------------------------  7  ---------------------------------------

# Predicción
X_test = test.copy()
preds = model.predict(X_test)
preds_labels = le.inverse_transform(preds)
# ------------------------  8  ---------------------------------------

# Generar archivo de submission
submission = pd.DataFrame({
    "ID": test["ID"],
    "RENDIMIENTO_GLOBAL": preds_labels
})
submission.to_csv("submission_gb_optimo.csv", index=False)
print("✅ Submission generado: submission_gb_optimo.csv")
# ------------------------  9  ---------------------------------------


✅ Submission generado: submission_gb_optimo.csv
